# 10 Minutes to cuDF and CuPy

This notebook provides introductory examples of how you can use cuDF and CuPy together to take advantage of CuPy array functionality (such as advanced linear algebra operations).

### Converting a cuDF DataFrame or Series to a CuPy Array

If we want to convert a cuDF `DataFrame` to a CuPy `ndarray`, the best way is to use the `dlpack` interface.

In [1]:
import time

import numpy as np
import cupy as cp
import cudf
from numba import cuda

In [2]:
nelem = 10000
df = cudf.DataFrame({'a':range(nelem),
                     'b':range(500, nelem + 500),
                     'c':range(1000, nelem + 1000)}
                   )

%time arr_cupy = cp.fromDlpack(df.to_dlpack())
type(arr_cupy)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 817 µs


/conda/envs/rapids/lib/python3.6/site-packages/cudf/io/dlpack.py:83: UserWarning: WARNING: cuDF to_dlpack() produces column-major (Fortran order) output. If the output tensor needs to be row major, transpose the output of this function.
  return cpp_dlpack.to_dlpack(gdf_cols)


cupy.core.core.ndarray

The best way to convert a cuDF `Series` to a CuPy `ndarray` is to either pass the underlying Numba `DeviceNDArray` to `cupy.asarray` or leverage the `dlpack` interface for conversions. We can also pass the `Series` itself, but this will be far slower.

In [3]:
col = 'a'

%time cola_cupy = cp.asarray(df[col].data.mem)
%time cola_cupy = cp.fromDlpack(df[col].to_dlpack())
%time cola_cupy = cp.asarray(df[col])
type(cola_cupy)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 287 µs
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 510 µs
CPU times: user 2.46 s, sys: 60 ms, total: 2.52 s
Wall time: 2.55 s


cupy.core.core.ndarray

From here, we can proceed with normal CuPy workflows, such as reshaping the array, getting the diagonal, or calculating the norm.

In [4]:
reshaped_arr = cola_cupy.reshape(50, 200)
reshaped_arr

array([[   0,    1,    2, ...,  197,  198,  199],
       [ 200,  201,  202, ...,  397,  398,  399],
       [ 400,  401,  402, ...,  597,  598,  599],
       ...,
       [9400, 9401, 9402, ..., 9597, 9598, 9599],
       [9600, 9601, 9602, ..., 9797, 9798, 9799],
       [9800, 9801, 9802, ..., 9997, 9998, 9999]])

In [5]:
reshaped_arr.diagonal()

array([   0,  201,  402,  603,  804, 1005, 1206, 1407, 1608, 1809, 2010,
       2211, 2412, 2613, 2814, 3015, 3216, 3417, 3618, 3819, 4020, 4221,
       4422, 4623, 4824, 5025, 5226, 5427, 5628, 5829, 6030, 6231, 6432,
       6633, 6834, 7035, 7236, 7437, 7638, 7839, 8040, 8241, 8442, 8643,
       8844, 9045, 9246, 9447, 9648, 9849])

In [6]:
cp.linalg.norm(reshaped_arr)

array(577306.967739)

### Converting a CuPy Array to a cuDF DataFrame or Series

We can also convert a CuPy `ndarray` to a cuDF `DataFrame` or `Series`. We can use the same `dlpack` interface from above, or rely on the `__cuda_array_interface__` and use cuDF's `from_gpu_matrix`. Either way, we'll need to make sure that our CuPy array is Fortran contiguous in memory (if it's not already). We can either transpose the array or simply coerce it to be Fortran contiguous beforehand.

We can check whether our array is Fortran contiguous by using `cp.isfortran` or looking at the [flags](https://docs-cupy.chainer.org/en/stable/reference/generated/cupy.ndarray.html#cupy.ndarray.flags) of the array.

In [7]:
cp.isfortran(reshaped_arr)

False

In this case, we'll need to convert it before going to cuDF.

In [8]:
reshaped_arr = cp.asfortranarray(reshaped_arr)
reshaped_df = cudf.from_dlpack(reshaped_arr.toDlpack())
print(reshaped_df.head())

/conda/envs/rapids/lib/python3.6/site-packages/cudf/io/dlpack.py:36: UserWarning: WARNING: cuDF from_dlpack() assumes column-major (Fortran order) input. If the input tensor is row-major, transpose it before passing it to this function.
  res, valids = cpp_dlpack.from_dlpack(pycapsule_obj)


     0    1    2    3    4    5    6 ...  199
0    0    1    2    3    4    5    6 ...  199
1  200  201  202  203  204  205  206 ...  399
2  400  401  402  403  404  405  406 ...  599
3  600  601  602  603  604  605  606 ...  799
4  800  801  802  803  804  805  806 ...  999
[192 more columns]


In [9]:
reshaped_df = cudf.DataFrame.from_gpu_matrix(reshaped_arr)
print(reshaped_df.head())

     0    1    2    3    4    5    6 ...  199
0    0    1    2    3    4    5    6 ...  199
1  200  201  202  203  204  205  206 ...  399
2  400  401  402  403  404  405  406 ...  599
3  600  601  602  603  604  605  606 ...  799
4  800  801  802  803  804  805  806 ...  999
[192 more columns]


If the array is not already contiguous, we'll need to create a contiguous array with `ascontiguousarray`.

In [10]:
diag_data = cp.ascontiguousarray(reshaped_arr.diagonal())
print(cudf.Series(diag_data))

0       0
1     201
2     402
3     603
4     804
5    1005
6    1206
7    1407
8    1608
9    1809
[40 more rows]
dtype: int64


### Interweaving CuDF and CuPy for Smooth PyData Workflows

RAPIDS libraries and the entire GPU PyData ecosystem are developing quickly, but sometimes one library may not have the functionality you need. One example of this might be taking the row-wise sum (or mean) of a Pandas DataFrame. cuDF's support for row-wise operations isn't mature, so you'd need to either transpose the DataFrame or write a UDF and explicitly calculate the sum across each row. Transposing could lead to hundreds of thousands of columns depending on your data size, which cuDF isn't optimized for currently, and writing a UDF can be a bit more time intensive.

By leveraging the interoperability of the GPU PyData ecosystem, this operation becomes very easy. Let's take the row-wise sum of our previously reshaped cuDF DataFrame.

In [11]:
print(reshaped_df.head())

     0    1    2    3    4    5    6 ...  199
0    0    1    2    3    4    5    6 ...  199
1  200  201  202  203  204  205  206 ...  399
2  400  401  402  403  404  405  406 ...  599
3  600  601  602  603  604  605  606 ...  799
4  800  801  802  803  804  805  806 ...  999
[192 more columns]


We can just transform it to a CuPy array via dlpack and use the `axis` argument of `sum`.

In [12]:
new_arr = cp.fromDlpack(reshaped_df.to_dlpack())
new_arr.sum(axis=1)

/conda/envs/rapids/lib/python3.6/site-packages/cudf/io/dlpack.py:83: UserWarning: WARNING: cuDF to_dlpack() produces column-major (Fortran order) output. If the output tensor needs to be row major, transpose the output of this function.
  return cpp_dlpack.to_dlpack(gdf_cols)


array([  19900,   59900,   99900,  139900,  179900,  219900,  259900,
        299900,  339900,  379900,  419900,  459900,  499900,  539900,
        579900,  619900,  659900,  699900,  739900,  779900,  819900,
        859900,  899900,  939900,  979900, 1019900, 1059900, 1099900,
       1139900, 1179900, 1219900, 1259900, 1299900, 1339900, 1379900,
       1419900, 1459900, 1499900, 1539900, 1579900, 1619900, 1659900,
       1699900, 1739900, 1779900, 1819900, 1859900, 1899900, 1939900,
       1979900])

With just that single line, we're able to seamlessly move between data structures in this ecosystem, giving us enormous flexibility without sacrificing speed.

### Converting a cuDF DataFrame to a CuPy Sparse Matrix.

We can also convert a `DataFrame` or `Series` to a CuPy sparse matrix. We might want to do this if downstream processes expect CuPy sparse matrices as an input.

The sparse matrix data structure is defined by three dense arrays, which we could create manually from an existing cuDF `DataFrame` or `Series`. Luckily, we don't need to do that. We can simply leverage `dlPack` again. We'll define a small helper function for cleanliness.

In [13]:
def cudf_to_cupy_sparse_matrix(data, sparseformat='column'):
    """Converts a cuDF object to a CuPy Sparse Column matrix.
    """
    if sparseformat not in ('row', 'column',):
        raise ValueError("Let's focus on column and row formats for now.")
    
    _sparse_constructor = cp.sparse.csc_matrix
    if sparseformat == 'row':
        _sparse_constructor = cp.sparse.csr_matrix

    return _sparse_constructor(cp.fromDlpack(data.to_dlpack()))

We can define a sparsely populated dataframe to illustrate this conversion to either sparse matrix format.

In [14]:
df = cudf.DataFrame()
nelem = 10000
nonzero = 1000
for i in range(20):
    arr = cp.random.normal(5, 5, nelem)
    arr[cp.random.choice(arr.shape[0], nelem-nonzero, replace=False)] = 0
    df['a' + str(i)] = arr

In [15]:
print(df.head())

    a0                 a1   a2   a3   a4               a5   a6 ...                a19
0  0.0                0.0  0.0  0.0  0.0  3.8952432357503  0.0 ...                0.0
1  0.0                0.0  0.0  0.0  0.0              0.0  0.0 ...                0.0
2  0.0                0.0  0.0  0.0  0.0              0.0  0.0 ...  8.126657540924068
3  0.0  7.130930187590361  0.0  0.0  0.0              0.0  0.0 ...                0.0
4  0.0                0.0  0.0  0.0  0.0              0.0  0.0 ...                0.0
[12 more columns]


In [17]:
sparse_data = cudf_to_cupy_sparse_matrix(df)

From here, we could continue our workflow with a CuPy sparse matrix.

For a full list of the functionality built into these libraries, we encourage you to check out the API docs for [cuDF](https://docs.rapids.ai/api/cudf/nightly/) and [CuPy](https://docs-cupy.chainer.org/en/stable/index.html).